In [192]:
import pandas as pd
from datetime import datetime, timedelta
import re
import requests
import json
import os

# Locial Structure

1. We download the real weather data and store it in a DataFrame
2. Loop over the forecasts
    - Take the forecast and restrict them to the next 2 days
    - Compare each forecast with the real weather and store the difference as a percentage
    - Add this percentage to a list. Now we can calculate the average

# 1. Download Real Weather Data

In [193]:
def create_date_and_hour(df:pd.DataFrame):
    df["date"] = pd.to_datetime(df.time).dt.date
    df["hour"] = pd.to_datetime(df.time).dt.hour
    return df

def get_weather(start_date:str, end_date:str) -> pd.DataFrame:
    api = f"https://archive-api.open-meteo.com/v1/archive?latitude=51.51&longitude=-0.13&start_date={start_date}&end_date={end_date}&hourly=temperature_2m,relativehumidity_2m,surface_pressure,direct_radiation,windspeed_10m,winddirection_10m"
    result = requests.get(api).text
    df = pd.DataFrame(json.loads(result).get("hourly"))

    # Create better readable date and hour
    df = create_date_and_hour(df)

    # Remove the last day, because it's empty
    df = df.loc[df.date != datetime.strptime(end_date, "%Y-%m-%d").date()]
    
    return df

In [194]:
# Right now the API takes 5 days to publish the historical weather data
real_weather = get_weather("2023-05-11", "2023-05-12")

# 2. Loop over forecasts

In [205]:
# Get all files in folder and store the filepaths
filepaths = []
# assign directory
directory = 'forecast-data'

# iterate over files in
# that directory
for filename in os.listdir(directory):
	f = os.path.join(directory, filename)
	# checking if it is a file
	if os.path.isfile(f):
		filepaths.append(f)

filepaths = sorted(filepaths)

In [212]:
for file in filepaths[:1]:
    # Remove [:1] when you want to do it on more data
    # Data is stored each day in a file. The only indicator which date we have is
    # the filename. We have to extract the date out of the filename in step 1

    # file name with extension
    forecast_date = os.path.basename(file).split("_")[0]
    forecast_date = datetime.strptime(forecast_date, "%Y-%m-%d").date()
    df = pd.read_json(file)
    df = create_date_and_hour(df)
    df = df.loc[(df["date"] > forecast_date) & (df["date"] <= forecast_date + timedelta(days=2))]
    # Now we merge the real weather data in the forecasts
    df = pd.merge(df, real_weather, on="time", how="inner")
    print(df.head())




,time,temperature_2m_x,relativehumidity_2m_x,surface_pressure_x,windspeed_10m_x,winddirection_10m_x,is_day,terrestrial_radiation,date_x,hour_x,temperature_2m_y,relativehumidity_2m_y,surface_pressure_y,direct_radiation,windspeed_10m_y,winddirection_10m_y,date_y,hour_y
